# 一等函数

在 Python 中，函数是一等对象。编程语言理论家把“一等对象”定义为满足下述条件的程序实体：

在运行时创建

能赋值给变量或数据结构中的元素

能作为参数传给函数

能作为函数的返回结果

有了一等函数，就可以使用函数式风格编程。函数式编程的特点之一是使用高阶函数


# 函数似为对象

python中几乎所有东西都是对象，对象拥有属性和方法

In [1]:
def factorial(n):
    """
    return n!
    """
    return 1 if n < 2 else n * factorial(n-1)

In [4]:
factorial(42)

1405006117752879898543142606244511569936384000000000

In [5]:
factorial.__doc__

'\n    return n!\n    '

In [7]:
print(type(factorial))

<class 'function'>


In [8]:
f = factorial

In [9]:
f

<function __main__.factorial(n)>

In [10]:
f(5)

120

In [11]:
list(map(f, range(6)))

[1, 1, 2, 6, 24, 120]

# 高阶函数

In [13]:
usernames = ["sophia", "emma", "olivia", "ava", "mia", "isabella", "zoe"]

In [14]:
sorted(usernames, key=len)

['ava', 'mia', 'zoe', 'emma', 'sophia', 'olivia', 'isabella']

In [15]:
sorted(usernames, key=lambda x: x[0])

['ava', 'emma', 'isabella', 'mia', 'olivia', 'sophia', 'zoe']

# 列表推导 vs map、filter、reduce

In [16]:
list(map(f, range(6)))

[1, 1, 2, 6, 24, 120]

In [17]:
[f(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [18]:
list(map(f, filter(lambda n: n % 2, range(6))))

[1, 6, 120]

In [19]:
[f(n) for n in range(6) if n % 2]

[1, 6, 120]

In [21]:
from functools import reduce
from operator import add

In [313]:
reduce(add, range(10))

45

# 可调用对象
不仅 Python 函数是真正的对象，任何 Python 对象都可以表现得像函数。为此，只需实现实例方法 __call__。

In [326]:
import random

class Surprise:

    def __init__(self, items):
        self._items = list(items) 
        random.shuffle(self._items)

    def pick(self): 
        try:
            return self._items.pop()
        except IndexError:
            raise 

    def __call__(self): 
        return self.pick()

In [327]:
Surprise(["dark chocolate", "milk", "white", "alcoholic"])()

'alcoholic'

# 接受任意参数的函数

## - 任意数量的位置参数

In [27]:
def avg(first, *rest):
    print(rest, type(rest))
    return (first + sum(rest)) / (1 + len(rest))
avg(1, 2, 3, 4)

(2, 3, 4) <class 'tuple'>


2.5

## - 任意数量的关键字参数

In [29]:
def kws_f(**kwargs):
    print(kwargs, type(kwargs))

In [30]:
kws_f(a=1, b=2, c=3)

{'a': 1, 'b': 2, 'c': 3} <class 'dict'>


## - 综合应用

In [88]:
# 生产HTML标签
def make_element(name, *contents, cls=None, **attrs):
    if cls:
        attrs['class'] = cls
        
    pairs = [f"{attr}={value}" for attr, value in attrs.items()]
    attr_str = ' '.join(pairs)
    
    if not contents:
        return f"<{name} {attr_str}/>"
    
    elements = [f"<{name} {attr_str}>{content}</{name}>" for content in contents]
    
    return '\n'.join(elements)
    
    

In [89]:
make_element('img', cls='pic-frame', src='a.jpg')

'<img src=a.jpg class=pic-frame/>'

In [90]:
print(make_element('item', *usernames, size='small', quantity='1'))

<item size=small quantity=1>sophia</item>
<item size=small quantity=1>emma</item>
<item size=small quantity=1>olivia</item>
<item size=small quantity=1>ava</item>
<item size=small quantity=1>mia</item>
<item size=small quantity=1>isabella</item>
<item size=small quantity=1>zoe</item>


# 支持函数式编程的包 functools和operator

https://docs.python.org/3/library/functools.html
https://docs.python.org/3/library/operator.html

In [52]:
# reduce函数和匿名函数实现阶乘计算
from functools import reduce
def fact(n):
    return reduce(lambda a, b: a * b, range(1, n+1))

In [53]:
fact(6)

720

In [46]:
from operator import mul

In [49]:
def fact(n):
    return reduce(mul, range(1, n+1))

In [54]:
fact(6)

720

In [56]:
# 使用functools.partial 冻结参数

In [62]:
from functools import partial

In [63]:
triple = partial(mul, 3)

In [64]:
triple(7)

21

# - 将partial 应用于 make_element上

In [91]:
picture = partial(make_element, 'img', cls='pic-frame')

In [92]:
picture(src='car.jpg')

'<img src=car.jpg class=pic-frame/>'

In [101]:
item = partial(make_element, 'item', size='large')

In [103]:
print(item('a', 'b', 'c'))

<item size=large>a</item>
<item size=large>b</item>
<item size=large>c</item>


# 解包与多返回值

任何序列或可迭代对象都可以通过一个简单的赋值操作来分解为单独的变量


In [105]:
def test():
    return 1, 2

In [106]:
a, b = test()

In [114]:
a, b, c = [1, 2, 3]

In [116]:
d1 = {"a": 1, "b": 2}
d2 = {"c": 3, "d": 4}

In [117]:
d = {**d1, **d2}

In [118]:
d

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

# 装饰器

In [119]:
# 举例: 我们想获得函数的运行时间

In [334]:
import time

def record_time(func):
    
    print("Run decorator.")
    
    def wrapper():
        start = time.time()
        result = func()
        end = time.time()
        print(func.__name__, end - start)
        return result
    return wrapper
    

In [335]:
def func_foo():
    time.sleep(2)
    print("Run func_foo.")

In [336]:
func = record_time(func_foo)

Run decorator.


In [337]:
func()

Run func_foo.
func_foo 2.0033421516418457


## - 装饰器语法糖@

In [154]:
@build_func_with_record
def func_bar():
    time.sleep(1)
    print("Run func_bar.")

In [155]:
func_bar()

Run func_bar.
func_bar 1.0034739971160889


## - 带参数

In [145]:
"""

重点强调：
- 装饰器一般来说不会修改函数的签名

- 也不会修改被包装函数的返回结果

- 使用*args, **kwargs是为了确保可以接受任何形式的输入参数

- 装饰器的返回值几乎总是同调用func(*args, **kwargs)的结果一致
"""

def record_time(func):
        
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end - start)
        return result
    return wrapper
    

In [178]:
@record_time
def make_element(name, *contents, cls=None, **attrs):
    """
    生成html标签元素
    """
    if cls:
        attrs['class'] = cls
        
    pairs = [f"{attr}={value}" for attr, value in attrs.items()]
    attr_str = ' '.join(pairs)
    
    if not contents:
        return f"<{name} {attr_str}/>"
    
    elements = [f"<{name} {attr_str}>{content}</{name}>" for content in contents]
    
    return '\n'.join(elements)

Run decorator.


In [172]:
make_element('img', cls='pic-frame', src='a.jpg')

'<img src=a.jpg class=pic-frame/>'

# - 保存元数据

In [189]:
"""
上述装饰器的使用，导致原始函数的元数据丢失
像函数名，文档字符串，调用签名等等
"""
make_element.__name__

'make_element'

In [180]:
make_element.__doc__

In [181]:
from inspect import signature

In [182]:
print(signature(make_element))

(*args, **kwargs)


In [184]:
# 引入@wraps
from functools import wraps

def record_time(func):
    
    """
    通过事先获取func的属性，并储存
    在返回的最终的函数中设置上述属性
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end - start)
        return result
    return wrapper

In [185]:
@record_time
def make_element(name, *contents, cls=None, **attrs):
    """
    生成html标签元素
    """
    if cls:
        attrs['class'] = cls
        
    pairs = [f"{attr}={value}" for attr, value in attrs.items()]
    attr_str = ' '.join(pairs)
    
    if not contents:
        return f"<{name} {attr_str}/>"
    
    elements = [f"<{name} {attr_str}>{content}</{name}>" for content in contents]
    
    return '\n'.join(elements)

In [186]:
make_element.__name__

'make_element'

In [187]:
make_element.__doc__

'\n    生成html标签元素\n    '

In [188]:
signature(make_element)

<Signature (name, *contents, cls=None, **attrs)>

## - 可接受参数的装饰器

In [191]:
"""
编写一个为函数添加日志功能的装饰器
允许用户指定日志的等级，以及一些其他的细节作为参数
"""

'\n编写一个为函数添加日志功能的装饰器\n允许用户指定日志的等级，以及一些其他的细节作为参数\n'

In [299]:
from functools import wraps
import logging
logging.basicConfig(format='%(asctime)s %(name)s %(levelname)s %(message)s')
logging.root.setLevel(logging.NOTSET)

def logged(level, name=None, message=None):
    
    def decorate(func):
        
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        
        logmsg = message if message else f"Function <{func.__name__}> is running."
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate
    

In [332]:
@logged(logging.DEBUG)
def make_element(name, *contents, cls=None, **attrs):
    """
    生成html标签元素
    """
    if cls:
        attrs['class'] = cls
        
    pairs = [f"{attr}={value}" for attr, value in attrs.items()]
    attr_str = ' '.join(pairs)
    
    if not contents:
        return f"<{name} {attr_str}/>"
    
    elements = [f"<{name} {attr_str}>{content}</{name}>" for content in contents]
    
    return '\n'.join(elements)

In [333]:
print(make_element('img', cls='pic-frame', src='a.jpg'))

2020-02-08 11:36:07,719 __main__ DEBUG Function <make_element> is running.


<img src=a.jpg class=pic-frame/>


## - 闭包

In [339]:
"""
闭包：指延伸了作用域的函数，其中包含函数定义体中引用、但是不在定义体中定义的非全局变量

关键是它能访问定义体之外定义的非全局变量

综上，闭包是一种函数，它会保留定义函数时存在的自由变量的绑定，这样调用函数时，虽然定义作用域不可用

但是仍能使用那些绑定


"""
# 基于保存历史的版本
def make_averager():
    # 在averager中 series 一般称为自由变量，指未在本地作用域中绑定的变量
    nums = []
    
    def averager(new_value):
        nums.append(new_value)
        total = sum(nums)
        return total / len(nums)
    return averager

In [340]:
avg = make_averager()

In [341]:
avg(1)

1.0

In [342]:
# 不保存历史数据

def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [343]:
avg = make_averager()

In [344]:
avg(1)

UnboundLocalError: local variable 'count' referenced before assignment

In [ ]:
"""
当count是不可变类型时，count += 1 其实是 count = count + 1，这个赋值操作会将count变为局部变量

但是之前的series作为列表是可变对象，不存在这个问题

对于不可变类型，只能读取，不能更新，count = count + 1其实会隐式的创建局部变量

引入nonlocal，作用是将变量标记为自由变量，即使函数中为变量赋予了新的值
"""

In [233]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [234]:
avg = make_averager()


1.0

In [236]:
avg(3)

2.0

In [306]:
# 动态设置装饰器属性

from functools import wraps
import logging
logging.basicConfig(format='%(asctime)s %(name)s %(levelname)s %(message)s')
logging.root.setLevel(logging.NOTSET)

def attach_wrapper(obj, func=None):
    if not func:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    
    def decorate(func):
        
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        
        logmsg = message if message else f"Function <{func.__name__}> is running."
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
    
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
            
        return wrapper
            
    return decorate

In [307]:
@logged(level=logging.INFO)
def hello():
    print("hello")

In [331]:
hello()

2020-02-08 11:35:36,915 __main__ INFO Function <hello> is running.


hello
hello 2.3126602172851562e-05


In [309]:
hello.set_level(logging.DEBUG)

In [330]:
hello()

2020-02-08 11:35:33,622 __main__ INFO Function <hello> is running.


hello
hello 3.504753112792969e-05


In [328]:
# 装饰器可以叠加使用
@logged(level=logging.INFO)
@record_time
def hello():
    print("hello")

In [329]:
hello()

2020-02-08 11:35:29,310 __main__ INFO Function <hello> is running.


hello
hello 0.00028586387634277344
